<h1><span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [45]:
import pyaudio
from pydub import AudioSegment

# Initialize pyaudio
p = pyaudio.PyAudio()

# Set the audio parameters
channels = 1
sample_rate = 44100
frame_size = 100

# Start recording
stream = p.open(format=pyaudio.paInt16,
                channels=channels,
                rate=sample_rate,
                input=True,
                frames_per_buffer=frame_size)

print("Recording...")
frames = []
for i in range(0, int(sample_rate / frame_size * 10)):
    data = stream.read(frame_size)
    frames.append(data)

# Stop recording
stream.stop_stream()
stream.close()
p.terminate()

print("Finished recording")

# Save the audio data as an mp3 file
audio_data = b''.join(frames)
audio = AudioSegment(
    audio_data,
    frame_rate=sample_rate,
    sample_width=2,
    channels=channels
)
audio.export("recording.mp3", format="mp3")


Recording...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/8w/r6kg1v9x7bbfzf9dw9gjslc80000gn/T/ipykernel_17338/172768589.py:22 in <module>     │
│                                                                                                  │
│                                                                                                  │
│ /Users/spencerbraun/.pyenv/versions/3.8.11/envs/start/lib/python3.8/site-packages/pyaudio/__init │
│ __.py:570 in read                                                                                │
│                                                                                                  │
│    569 │   │   │   │   │   │   │     paCanNotReadFromAnOutputOnlyStream)                         │
│ ❱  570 │   │   │   return pa.read_stream(self._stream, num_frames,                               │
│    571 │   │   │   │   │   │   │   │     exception_on_overflow)                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [1]:
import whisper
model = whisper.load_model("small")

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("recording.mp3")

100%|███████████████████████████████████████| 461M/461M [00:16<00:00, 29.9MiB/s]


In [29]:
audio

In [24]:
import whisper
import modal
import gradio as gr
from pathlib import Path
stub = modal.Stub("whisper-transcribe")
app_image = (
    modal.Image.debian_slim()
    .pip_install(
        "https://github.com/openai/whisper/archive/9f70a352f9f8630ab3aa0d06af5cb9532bd8c21d.tar.gz",
        "dacite",
        "jiwer",
        "ffmpeg-python",
        "gql[all]~=3.0.0a5",
        "pandas",
        "loguru==0.6.0",
        "torchaudio==0.12.1",
        "pydub"
    )
    .apt_install("ffmpeg")
    .pip_install("ffmpeg-python")
)

volume = modal.SharedVolume().persist("whisper-modal")
MODEL_DIR = Path("/model")

@stub.function(
    gpu="t4",
    image=app_image,
    timeout=100,
    shared_volumes={
        str(MODEL_DIR): volume,  # fine-tuned model will be stored at `MODEL_DIR`
    },
)
def transcribe(audio):
    import torch
    use_gpu = torch.cuda.is_available()
    device = "cuda" if use_gpu else "cpu"
    model = whisper.load_model("small", device=device, download_root=MODEL_DIR)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")

    # decode the audio
    options = whisper.DecodingOptions(fp16 = False)
    result = whisper.decode(model, mel, options)
    return result.text

In [30]:
audio = whisper.load_audio("recording.mp3")
with stub.run():
    result = transcribe.call(audio)

Output()

✓ Initialized. View app at https://modal.com/apps/ap-OMDStZ1unCqv4Ep3CGC21R

Output()

✓ Created objects.
└── 🔨 Created transcribe.

Output()

Detected language: en
⠋ Running (1/1 containers active)...

✓ App completed.

In [31]:
result

"testing, recording, and I'm still talking, and it's just open for a frame size of 100."

In [36]:
stop_listening??

result2:
{   'alternative': [   {   'confidence': 0.74288338,
                           'transcript': 'right exciting attention'},
                       {'transcript': 'exciting attention'},
                       {'transcript': 'right excited attention'},
                       {'transcript': 'excited attention'},
                       {'transcript': "right I think I'm excited attention"}],
    'final': True}
Google Speech Recognition thinks you said right exciting attention


In [43]:
r.recognize_whisper

<bound method Recognizer.recognize_whisper of <speech_recognition.Recognizer object at 0x2c7538ca0>>

In [44]:

import time

import speech_recognition as sr


# this is called from the background thread
def callback(recognizer, audio):
    # received audio data, now we'll recognize it using Google Speech Recognition
    try:
        # for testing purposes, we're just using the default API key
        # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
        # instead of `r.recognize_google(audio)`
        print("Google Speech Recognition thinks you said " + recognizer.recognize_google(audio))
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))


r = sr.Recognizer()
m = sr.Microphone()
with m as source:
    r.adjust_for_ambient_noise(source)  # we only need to calibrate once, before we start listening

# start listening in the background (note that we don't have to do this inside a `with` statement)
stop_listening = r.listen_in_background(m, callback)
# `stop_listening` is now a function that, when called, stops background listening

# do some unrelated computations for 5 seconds
for _ in range(50): time.sleep(0.1)  # we're still listening even though the main thread is doing other things

# calling this function requests that the background listener stop listening
stop_listening(wait_for_stop=False)

# do some more unrelated things
while True: time.sleep(0.1)  

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/8w/r6kg1v9x7bbfzf9dw9gjslc80000gn/T/ipykernel_17338/2189133450.py:36 in <module>    │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [18]:

    
import gradio as gr

gr.Interface(
    title = 'OpenAI Whisper ASR Gradio Web UI', 
    fn=transcribe, 
    inputs=[
        gr.inputs.Audio(source="microphone", type="filepath")
    ],
    outputs=[
        "textbox"
    ],
    live=True).launch()


/Users/spencerbraun/.pyenv/versions/3.8.11/envs/start/lib/python3.8/site-packages/gradio/inputs.py:319: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/Users/spencerbraun/.pyenv/versions/3.8.11/envs/start/lib/python3.8/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


/Users/spencerbraun/.pyenv/versions/3.8.11/envs/start/lib/python3.8/site-packages/gradio/processing_utils.py:236: UserWarning: Trying to convert audio automatically from int32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
/Users/spencerbraun/.pyenv/versions/3.8.11/envs/start/lib/python3.8/site-packages/anyio/_backends/_asyncio.py:867: DeprecationError: 2022-12-05: Calling a function directly is deprecated. Use f.call(...) instead. In a future version of Modal, f(...) will be used to call a function in the same process.
  result = context.run(func, *args)
Traceback (most recent call last):
  File "/Users/spencerbraun/.pyenv/versions/3.8.11/envs/start/lib/python3.8/site-packages/gradio/routes.py", line 337, in run_predict
    output = await app.get_blocks().process_api(
  File "/Users/spencerbraun/.pyenv/versions/3.8.11/envs/start/lib/python3.8/site-packages/gradio/blocks.py", line 1015, in process_api
    result = await self.call_function(
  File "/Users/spencerbraun